<a href="https://colab.research.google.com/github/pansplawik/eegParkinsonCognitiveDisorders/blob/master/pracaInzynierska.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mne

In [59]:
import mne
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import csv

In [52]:
labels = pd.read_csv('/content/drive/MyDrive/dyplom/dataset.csv', sep=';')
path='/content/drive/MyDrive/dyplom/EEG'

In [56]:
def load_epochs_from_dir(path, labels):
    epochs_array = []
    for filename, label in zip(os.listdir(path), labels['PD']):
        if filename.endswith('.edf'):
            filepath = os.path.join(path, filename)
            raw = mne.io.read_raw_edf(filepath, preload=True)
            events = mne.make_fixed_length_events(raw, duration=1.0)
            raw.filter(1, 30, fir_design='firwin')
            epochs = mne.Epochs(raw, events, tmin=0, tmax=2, baseline=(0,2), detrend=1,reject_by_annotation='break')
            epochs_array.append([epochs, label])
            del epochs
            del raw
    return epochs_array


In [83]:
raw = mne.io.read_raw_edf('PD129.edf', preload=True)
events = mne.make_fixed_length_events(raw, duration=1.0)
raw.filter(1, 30, fir_design='firwin')
epochs = mne.Epochs(raw, events, tmin=0, tmax=2, baseline=(0,2), detrend=1,reject_by_annotation='nan')
print(epochs.drop_log)

Extracting EDF parameters from /content/PD129.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 755199  =      0.000 ...  1474.998 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 1691 samples (3.303 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s


Not setting metadata
1475 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
((), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), (), 

[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.6s finished


In [ ]:
def psds(epochs_array, labels):
    data = []
    for epochs, label in zip(epochs_array, labels):
        epo_spectrum = epochs.compute_psd(method='multitaper', fmin=8, fmax=13, picks='eeg')
        psds, freqs = epo_spectrum.get_data(return_freqs=True)
        for p in psds:
            data.append([p, label])
        del psds
        del freqs
    return data

In [ ]:
data = load_epochs_from_dir(path,labels)
df = pd.DataFrame(data, columns=['value', 'label'])
df.to_csv('data.csv', index=False)

In [ ]:
psd = psds([epochs for epochs, label in data], [label for epochs, label in data])
df = pd.DataFrame(psd, columns=['psds', 'label'])
df.to_csv('psd.csv', index=False)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split([x[0] for x in psd], [x[1] for x in psd], test_size=0.2, random_state=42)
X_train_2d = np.array([x[0] for x in X_train]).reshape(len(X_train), -1)
X_test_2d = np.array([x[0] for x in X_test]).reshape(len(X_test), -1)
y_train=np.array(y_train)
y_train=y_train.transpose()
y_test=np.array(y_test)
y_test=y_test.transpose()
svm = SVC(kernel='linear', C=1, random_state=42)

In [ ]:
print(X_test_2d.shape)
print(X_train_2d.shape)

In [ ]:
svm.fit(X_train_2d, y_train)

In [ ]:
y_pred = svm.predict(X_test_2d)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
